In [ ]:
!pip install trl==0.11.3 -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.1 MB/s eta 0:00:00


In [ ]:
import torch
import json
import os
import numpy as np
import random
import re
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelForSequenceClassification
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from datasets import Dataset as HFDataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import wandb
import time
from datetime import datetime

In [ ]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ericzhangez1006 (ericzhangez1006-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import json
import os
import re
import time
import random
import wandb
from datetime import datetime
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import Dataset as HFDataset
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from sentence_transformers import SentenceTransformer, util

# ====================== Custom PPO Trainer ======================
class CustomPPOTrainer(PPOTrainer):
    """Custom PPO Trainer that prevents data shuffling to maintain alignment"""

    def prepare_dataloader(self, dataset, data_collator):
        """Override prepare_dataloader to disable shuffling"""
        return DataLoader(
            dataset,
            batch_size=self.config.batch_size,
            shuffle=False,  # Disable shuffling
            collate_fn=data_collator,
            drop_last=False,
            pin_memory=True,
            num_workers=0,
        )

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Emotion Extraction ======================
def extract_emotion_and_text_v4(full_response):
    """Extract emotion and text from SFT format: 'therapist_text <therapist_emotion> emotion_word <eos>'"""
    full_response = full_response.strip()

    if full_response.endswith('<eos>'):
        full_response = full_response[:-5].strip()

    emotion_pattern = r'<therapist_emotion>'
    emotion_matches = list(re.finditer(emotion_pattern, full_response))

    if emotion_matches:
        last_emotion_match = emotion_matches[-1]
        last_emotion_start = last_emotion_match.end()

        emotion_part = full_response[last_emotion_start:].strip()

        therapist_text = full_response[:last_emotion_match.start()].strip()
        emotion_word = re.sub(r'<eos>.*$', '', emotion_part).strip().lower()

        emotion_words = emotion_word.split()
        if emotion_words:
            emotion_word = emotion_words[0]

        valid_emotions = {"anger", "joy", "neutral", "sadness", "depression", "disgust", "fear"}
        if emotion_word in valid_emotions:
            return therapist_text, emotion_word, True
        else:
            return therapist_text, emotion_word, True

    return full_response, "", False

# ====================== Advanced Reward Calculator ======================
class TherapyRewardCalculator:
    """Enhanced reward calculator with sentiment and empathy models"""

    def __init__(self, device):
        self.device = device

        # 1. Sentiment classifier (distillBERT)
        self.sentiment_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
        self.sentiment_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english").to(device)

        # 2. Empathy classifier (bert_empathy)
        self.empathy_tokenizer = AutoTokenizer.from_pretrained("paragon-analytics/bert_empathy")
        self.empathy_model = AutoModelForSequenceClassification.from_pretrained("paragon-analytics/bert_empathy").to(device)

    def compute_empathy_reward(self, text):
        """Component 4: bert_empathy score"""
        try:
            inputs = self.empathy_tokenizer(text, return_tensors="pt",
                                          truncation=True, padding=True, max_length=512).to(self.device)

            with torch.no_grad():
                outputs = self.empathy_model(**inputs)
                probs = torch.softmax(outputs.logits, dim=-1)
                empathy_score = probs[0][1].item()

            return empathy_score
        except:
            return 0.5

    def compute_sentiment_reward(self, text):
        """Component 2: distillBERT sentiment (positive sentiment reward)"""
        try:
            inputs = self.sentiment_tokenizer(text, return_tensors="pt",
                                            truncation=True, padding=True, max_length=512).to(self.device)

            with torch.no_grad():
                outputs = self.sentiment_model(**inputs)
                probs = torch.softmax(outputs.logits, dim=-1)
                positive_score = probs[0][1].item()  

            return positive_score
        except:
            return 0.5 

# ====================== Dataset Class ======================
class TherapyDatasetV4(Dataset):
    """Dataset class for 4thFIXED preprocessed data"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path and os.path.exists(tokenizer_path):
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
            print(f"Loaded tokenizer from {tokenizer_path}")
        else:
            raise ValueError(f"Tokenizer not found at {tokenizer_path}")

        self.bos_id = self.tokenizer.bos_token_id
        self.eos_id = self.tokenizer.eos_token_id
        self.pad_id = self.tokenizer.pad_token_id
        self.problem_id = self.tokenizer.convert_tokens_to_ids("<problem>")
        self.user_id = self.tokenizer.convert_tokens_to_ids("<user>")
        self.user_emotion_id = self.tokenizer.convert_tokens_to_ids("<user_emotion>")
        self.therapist_id = self.tokenizer.convert_tokens_to_ids("<therapist>")
        self.therapist_emotion_id = self.tokenizer.convert_tokens_to_ids("<therapist_emotion>")

        self.max_length = 128
        self.processed_data = []
        self._preprocess_data_v4()

    def _preprocess_data_v4(self):
        """Preprocess dialog data using 4thFIXED format"""
        for conv in tqdm(self._data, desc="Processing data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    input_encoding = self.tokenizer(
                        input_text,
                        max_length=self.max_length,
                        padding='max_length',
                        truncation=True,
                        return_tensors='pt'
                    )

                    self.processed_data.append({
                        'input_ids': input_encoding['input_ids'].squeeze(),
                        'attention_mask': input_encoding['attention_mask'].squeeze(),
                        'query': input_text,
                        'target_text': therapist_text,
                        'target_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion
                    })

                    user_text_parts = []
                    user_emotions = []

        print(f"Processed {len(self.processed_data)} therapist responses")

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]

    def to_hf_dataset(self):
        """Convert to HuggingFace Dataset format"""
        return HFDataset.from_dict({
            'input_ids': [x['input_ids'].tolist() for x in self.processed_data],
            'attention_mask': [x['attention_mask'].tolist() for x in self.processed_data],
            'query': [x['query'] for x in self.processed_data],
            'target_text': [x['target_text'] for x in self.processed_data],
            'target_emotion': [x['target_emotion'] for x in self.processed_data],
            'user_input': [x['user_input'] for x in self.processed_data],
            'user_emotion': [x['user_emotion'] for x in self.processed_data]
        })

# ====================== Generation Function ======================
def generate_therapy_response(model, tokenizer, query_tensor, device, max_new_tokens=128):
    """Generate therapy response with conservative parameters"""

    generation_kwargs = {
        "do_sample": True,
        "top_p": 1.0,
        "top_k": 0.0,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "max_new_tokens": max_new_tokens,
    }

    with torch.no_grad():
        outputs = model.generate(
            query_tensor.unsqueeze(0).to(device),
            **generation_kwargs
        )

    generated_ids = outputs[0].cpu()
    new_tokens = generated_ids[len(query_tensor):]

    full_response = tokenizer.decode(
        new_tokens.tolist(),
        skip_special_tokens=False,
        clean_up_tokenization_spaces=True
    ).strip()

    therapist_text, therapist_emotion, has_emotion = extract_emotion_and_text_v4(full_response)

    return new_tokens.to(device), therapist_text, therapist_emotion, has_emotion

# ====================== ENHANCED Reward Functions ======================
def detect_local_ngram_repetition(text, n=2, window_size=10, max_repeats=1, weight=3.0):
    """Enhanced repetition detection with stricter penalties"""
    words = text.split()
    if len(words) < n:
        return 0.0

    penalty = 0.0
    for i in range(len(words) - window_size + 1):
        window = words[i:i + window_size]
        ngrams = [tuple(window[j:j + n]) for j in range(len(window) - n + 1)]

        ngram_counts = {}
        for ngram in ngrams:
            ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1

        for ngram, count in ngram_counts.items():
            if count > max_repeats:
                penalty += (count - max_repeats) ** 1.5 * weight

    return penalty

def compute_text_quality_score_enhanced(response):
    """Text quality scoring with enhanced penalties for poor responses"""
    if not response or not response.strip():
        return -10.0  # Stricter penalty for empty responses

    response_clean = response.strip()
    words = response_clean.split()

    base_score = 2.0  # Baseline score

    # Enhanced repetition penalties
    bigram_penalty = detect_local_ngram_repetition(response_clean, n=2, window_size=6, max_repeats=1, weight=4.0)
    trigram_penalty = detect_local_ngram_repetition(response_clean, n=3, window_size=5, max_repeats=1, weight=6.0)

    # Word-level repetition penalty
    word_penalty = 0.0
    word_counts = {}
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word.lower())
        if len(clean_word) > 3:
            word_counts[clean_word] = word_counts.get(clean_word, 0) + 1

    for token, count in word_counts.items():
        if count > 4:  
            # Exponential penalty: more repeats = heavier punishment
            penalty = min((count - 1) ** 1.7 * 1.5, 8.0)
            word_penalty -= penalty

    # Meaningless pattern detection (enhanced)
    meaningless_patterns = [
        r'^[^a-zA-Z]*$',      # No letters
        r'^\W+$',             # All symbols
        r'^(.)\1{4,}$',       # Repeated characters (lower threshold)
        r'[^\x00-\x7F]{3,}',  # Non-standard characters (lower threshold)
        r'\b(?:yes|no|ok|thanks?)\b',  # Simple responses
        r'.{100,}'            # Too long responses
    ]

    for pattern in meaningless_patterns:
        if re.search(pattern, response_clean, re.IGNORECASE):
            return -10.0  # Stricter penalty for meaningless responses

    quality_score = base_score - bigram_penalty - trigram_penalty + word_penalty
    return max(min(quality_score, 8.0), -10.0)  # Extended negative range

def compute_emotion_score_enhanced(has_emotion_token, generated_emotion, target_emotion):
    """Emotion scoring with enhanced rewards for non-neutral matches"""
    if not has_emotion_token:
        return -7.0  # Enhanced penalty for missing emotion tokens

    valid_emotions = {"anger", "joy", "neutral", "sadness", "depression", "disgust", "fear"}
    generated_emotion = generated_emotion.lower()

    score = 0.0

    # Stricter penalty for invalid emotions
    if generated_emotion not in valid_emotions:
        return -6.0

    score += 1.0  # Base score for valid emotions

    if not target_emotion:
        return score

    target_emotion = target_emotion.lower()

    # Emotion group definitions
    emotion_groups = {
        "negative": {"sadness", "depression", "anger", "disgust", "fear"},
        "positive": {"joy"},
        "neutral": {"neutral"}
    }

    gen_group = next((g for g, e in emotion_groups.items() if generated_emotion in e), None)
    target_group = next((g for g, e in emotion_groups.items() if target_emotion in e), None)

    # Exact match rewards (enhanced for non-neutral emotions)
    if generated_emotion == target_emotion:
        if gen_group != "neutral":
            return score + 6.0  
        else:
            return score + 1.0  # Neutral exact match reward

    # Group match rewards (enhanced for non-neutral emotions)
    if gen_group and target_group and gen_group == target_group:
        if gen_group != "neutral":
            return score + 3.0  # Non-neutral group match reward
        else:
            return score + 1.0  # Neutral group match reward

    # Strict penalty for emotional conflicts
    if (gen_group == "positive" and target_group == "negative") or \
       (gen_group == "negative" and target_group == "positive"):
        return -5.0

    return score - 1.0  # Base penalty for mismatches

def compute_contextual_relevance_enhanced(generated_text, user_input, similarity_model, device):
    """Contextual relevance with enhanced penalties for irrelevance"""
    if not generated_text or not user_input:
        return -8.0  # Enhanced penalty for missing inputs

    try:
        # Text cleaning
        gen_clean = re.sub(r'[^\w\s]', ' ', generated_text.lower()).strip()
        user_clean = re.sub(r'[^\w\s]', ' ', user_input.lower()).strip()

        if not gen_clean or not user_clean:
            return -5.0

        # Semantic similarity calculation
        embeddings = similarity_model.encode([gen_clean, user_clean], convert_to_tensor=True)
        embeddings = embeddings.to(device)
        cos_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1])
        similarity = float(cos_sim.item())

        # Enhanced penalty mechanism
        if similarity < 0.03:
            return -10.0  
        elif similarity < 0.05:
            return -8.0
        elif similarity < 0.08:
            return -6.0
        elif similarity < 0.12:
            return -3.0
        elif similarity < 0.20:
            return 0.0
        elif similarity < 0.30:
            return 2.0
        elif similarity < 0.50:
            return 4.0
        elif similarity < 0.70:
            return 6.0
        else:
            return 8.0

    except Exception as e:
        print(f"Error in relevance calculation: {e}")
        return -2.0  

def compute_comprehensive_rewards_enhanced(
    text_responses,
    emotion_responses,
    has_emotion_flags,
    target_texts,
    target_emotions,
    user_inputs,
    user_emotions,
    similarity_model,
    reward_calculator,
    device
):
    """Enhanced reward function with wider dynamic range (-12 to 10)"""
    rewards = []

    for i, (text, emotion, has_emotion, target_text, target_emotion) in enumerate(zip(
        text_responses, emotion_responses, has_emotion_flags, target_texts, target_emotions
    )):
        quality_score = compute_text_quality_score_enhanced(text)
        emotion_score = compute_emotion_score_enhanced(has_emotion, emotion, target_emotion)

        user_input = user_inputs[i] if i < len(user_inputs) else ""
        relevance_score = compute_contextual_relevance_enhanced(text, user_input, similarity_model, device)

        # Sentiment and empathy scores (narrowed range)
        if reward_calculator and text.strip():
            sentiment_score = reward_calculator.compute_sentiment_reward(text) * 1.1 - 0.5  # [-0.5, 1.0]
            empathy_score = reward_calculator.compute_empathy_reward(text) * 1.1 - 0.5      # [-0.5, 1.0]
        else:
            sentiment_score = 0.0
            empathy_score = 0.0

        # New penalty mechanisms
        penalty = 0.0

        # Emotional conflict penalty (user emotion vs response emotion)
        if user_emotions and i < len(user_emotions):
            user_emo = user_emotions[i].lower()
            if (user_emo in ["sadness", "depression", "anger", "disgust", "fear"] and
                emotion.lower() in ["joy"]):
                penalty -= 1.5

        # Balanced weighted calculation
        total_reward = (
            quality_score * 1.1 +        # Increased quality weight
            emotion_score * 1.2 +
            relevance_score * 1.1 +      # Increased relevance weight
            sentiment_score * 0.7 +      # Reduced sentiment weight
            empathy_score * 0.7 +        # Reduced empathy weight
            penalty                      # New penalty term
        )

        bounded_reward = max(min(total_reward, 10.0), -10.0)
        rewards.append(torch.tensor(bounded_reward, dtype=torch.float32))

    return rewards

# ====================== Legacy Function Names (for compatibility) ======================
def compute_text_quality_score_fixed(response):
    """Legacy wrapper for enhanced text quality scoring"""
    return compute_text_quality_score_enhanced(response)

def compute_emotion_score(has_emotion_token, generated_emotion, target_emotion):
    """Legacy wrapper for enhanced emotion scoring"""
    return compute_emotion_score_enhanced(has_emotion_token, generated_emotion, target_emotion)

def compute_contextual_relevance_score_progressive(generated_text, user_input, similarity_model, device):
    """Legacy wrapper for enhanced contextual relevance scoring"""
    return compute_contextual_relevance_enhanced(generated_text, user_input, similarity_model, device)

# ====================== Data Collator ======================
def ppo_collator(batch):
    """PPO data collator"""
    return {
        'input_ids': torch.stack([torch.tensor(item['input_ids'], dtype=torch.long) for item in batch]),
        'attention_mask': torch.stack([torch.tensor(item['attention_mask'], dtype=torch.long) for item in batch]),
    }

# ====================== Weights & Biases Logging ======================
def log_sample_conversations(text_responses, user_inputs, batch_idx, epoch, max_samples=3):
    """Log sample conversation pairs to W&B"""
    conversations = []
    for i in range(min(len(text_responses), max_samples)):
        conversations.append({
            "epoch": epoch,
            "batch": batch_idx,
            "sample_id": i,
            "user_input": user_inputs[i],
            "therapist_response": text_responses[i]
        })

    table = wandb.Table(
        columns=["epoch", "batch", "sample_id", "user_input", "therapist_response"],
        data=[[conv["epoch"], conv["batch"], conv["sample_id"],
               conv["user_input"], conv["therapist_response"]] for conv in conversations]
    )

    wandb.log({"sample_conversations": table})

def log_score_distributions(scores_dict, step):
    """Log score distributions as histograms"""
    for name, scores in scores_dict.items():
        if scores and len(scores) > 0:
            wandb.log({
                f"distributions/{name}": wandb.Histogram(scores),
                f"distributions/{name}_mean": np.mean(scores),
                f"distributions/{name}_std": np.std(scores)
            }, step=step)

# ====================== Main Training Function ======================
def run_ppo_training_with_wandb(
    train_data_path,
    val_data_path,
    sft_model_path,
    tokenizer_path,
    num_epochs=3,
    batch_size=8,
    save_dir="ppo_model_with_wandb",
    project_name="therapy-ppo-training",
    run_name=None
):
    """Run PPO training with comprehensive W&B logging"""

    print("Starting PPO training with Weights & Biases integration...")

    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    os.makedirs(save_dir, exist_ok=True)

    if run_name is None:
        run_name = f"8m18therapy_ppo_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    wandb.init(
        project=project_name,
        name=run_name,
        config={
            "architecture": "GPT2",
            "dataset": "therapy_conversations",
            "num_epochs": num_epochs,
            "batch_size": batch_size,
            "max_length": 128,
            "max_new_tokens": 128,
            "relevance_function": "enhanced_semantic_scoring"
        }
    )

    print(f"W&B run: {wandb.run.name}")
    print(f"W&B url: {wandb.run.url}")

    global_step = 0

    # Load tokenizer
    print("Loading tokenizer...")
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    print(f"Tokenizer vocabulary size: {len(tokenizer)}")

    # Load base model
    print("Loading base model...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))

    # Load SFT checkpoint
    print(f"Loading SFT checkpoint from {sft_model_path}...")
    try:
        checkpoint = torch.load(sft_model_path, map_location='cpu', weights_only=False)

        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        elif 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint

        cleaned_state_dict = {}
        for key, value in state_dict.items():
            if key.startswith('model.'):
                cleaned_state_dict[key[6:]] = value
            else:
                cleaned_state_dict[key] = value

        model.load_state_dict(cleaned_state_dict, strict=False)
        model.to(device)
        print("SFT checkpoint loaded successfully")

    except Exception as e:
        print(f"Error loading SFT checkpoint: {e}")
        print("Continuing with base model...")

    # Initialize similarity model (REQUIRED)
    print("Loading similarity model...")
    try:
        similarity_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        print("Similarity model loaded successfully")
    except Exception as e:
        print(f"CRITICAL ERROR: Failed to load similarity model: {e}")
        print("Similarity model is required for relevance scoring. Training cannot continue.")
        raise RuntimeError("Similarity model is required but failed to load")

    # Initialize enhanced reward calculator with sentiment and empathy models
    print("Loading sentiment and empathy models...")
    try:
        reward_calculator = TherapyRewardCalculator(device)
        print("Sentiment and empathy models loaded successfully")
    except Exception as e:
        print(f"Error loading sentiment/empathy models: {e}")
        reward_calculator = None

    # Create PPO configuration
    print("Creating PPO configuration...")

    ppo_config = PPOConfig(
        model_name='gpt2',
        whiten_rewards=True,
        learning_rate=1e-6,
        batch_size=batch_size,
        mini_batch_size=batch_size,
        gradient_accumulation_steps=1,
        seed=42,
    )

    # Log PPO hyperparameters to W&B
    wandb.config.update({
        "ppo_learning_rate": ppo_config.learning_rate,
        "ppo_batch_size": ppo_config.batch_size,
        "ppo_whiten_rewards": ppo_config.whiten_rewards,
        "generation_top_p": 1.0,
        "generation_top_k": 0.0,
        "enhanced_rewards": "comprehensive_enhanced_scoring",
        "quality_function": "enhanced_repetition_detection",
        "relevance_function": "enhanced_semantic_similarity",
        "data_alignment": "custom_trainer_no_shuffle"
    })

    # Create PPO models
    print("Creating PPO models...")
    ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model,
        torch_dtype=torch.float32
    )
    ppo_model.to(device)

    ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model,
        torch_dtype=torch.float32
    )
    ref_model.to(device)

    # Load datasets
    print("Loading datasets...")
    train_therapy_dataset = TherapyDatasetV4(train_data_path, tokenizer_path)
    val_therapy_dataset = TherapyDatasetV4(val_data_path, tokenizer_path)

    full_train_dataset = train_therapy_dataset.to_hf_dataset()
    full_val_dataset = val_therapy_dataset.to_hf_dataset()

    # Extract metadata
    train_target_texts = full_train_dataset['target_text']
    train_target_emotions = full_train_dataset['target_emotion']
    train_queries = full_train_dataset['query']
    train_user_inputs = full_train_dataset['user_input']
    train_user_emotions = full_train_dataset['user_emotion']

    val_target_texts = full_val_dataset['target_text']
    val_target_emotions = full_val_dataset['target_emotion']
    val_user_inputs = full_val_dataset['user_input']
    val_user_emotions = full_val_dataset['user_emotion']

    # Create PPO datasets
    train_dataset = full_train_dataset.remove_columns([
        'target_text', 'target_emotion', 'query', 'user_input', 'user_emotion'
    ])
    val_dataset = full_val_dataset.remove_columns([
        'target_text', 'target_emotion', 'query', 'user_input', 'user_emotion'
    ])

    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")

    # Log dataset info to W&B
    wandb.config.update({
        "train_dataset_size": len(train_dataset),
        "val_dataset_size": len(val_dataset),
        "total_parameters": sum(p.numel() for p in ppo_model.parameters()),
        "trainable_parameters": sum(p.numel() for p in ppo_model.parameters() if p.requires_grad)
    })

    # Initialize CUSTOM PPO trainer (prevents data shuffling)
    print("Initializing Custom PPO trainer...")
    ppo_trainer = CustomPPOTrainer(
        config=ppo_config,
        model=ppo_model,
        ref_model=ref_model,
        tokenizer=tokenizer,
        dataset=train_dataset,
        data_collator=ppo_collator
    )

    # Training loop
    print("Starting PPO training...")
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        epoch_start_time = time.time()

        # Epoch metrics
        epoch_rewards = []
        epoch_quality_scores = []
        epoch_emotion_scores = []
        epoch_relevance_scores = []
        epoch_sentiment_scores = []
        epoch_empathy_scores = []
        epoch_emotion_success = []

        for batch_idx, batch in enumerate(tqdm(ppo_trainer.dataloader, desc="Training batches")):
            try:
                query_tensors = [item.to(device) for item in batch["input_ids"]]

                # Get current batch data with perfect alignment (no shuffling)
                start_idx = batch_idx * batch_size
                end_idx = min(start_idx + batch_size, len(train_target_texts))
                current_batch_size = end_idx - start_idx

                # Skip if we're beyond the data bounds
                if start_idx >= len(train_target_texts):
                    continue

                # Extract metadata for current batch
                batch_target_texts = train_target_texts[start_idx:end_idx]
                batch_target_emotions = train_target_emotions[start_idx:end_idx]
                batch_user_inputs = train_user_inputs[start_idx:end_idx]
                batch_user_emotions = train_user_emotions[start_idx:end_idx]

                # Ensure tensor batch size matches metadata batch size
                query_tensors = query_tensors[:current_batch_size]

                # Generate responses
                response_tensors = []
                text_responses = []
                emotion_responses = []
                has_emotion_flags = []

                for query_tensor in query_tensors:
                    response_tokens, therapist_text, therapist_emotion, has_emotion = generate_therapy_response(
                        ppo_model.pretrained_model,
                        tokenizer,
                        query_tensor,
                        device
                    )
                    response_tensors.append(response_tokens)
                    text_responses.append(therapist_text)
                    emotion_responses.append(therapist_emotion)
                    has_emotion_flags.append(has_emotion)

                # Compute rewards with ENHANCED scoring
                rewards = compute_comprehensive_rewards_enhanced(
                    text_responses,
                    emotion_responses,
                    has_emotion_flags,
                    batch_target_texts,
                    batch_target_emotions,
                    batch_user_inputs,
                    batch_user_emotions,
                    similarity_model,
                    reward_calculator,
                    device
                )

                # Compute individual component scores for detailed logging
                batch_quality_scores = [compute_text_quality_score_enhanced(text) for text in text_responses]
                batch_emotion_scores = [
                    compute_emotion_score_enhanced(has_emotion, emotion, target_emotion)
                    for has_emotion, emotion, target_emotion in zip(has_emotion_flags, emotion_responses, batch_target_emotions)
                ]
                batch_relevance_scores = [
                    compute_contextual_relevance_enhanced(text, user_input, similarity_model, device)
                    for text, user_input in zip(text_responses, batch_user_inputs)
                ]

                # Track sentiment and empathy scores
                batch_sentiment_scores = [
                    reward_calculator.compute_sentiment_reward(text) if reward_calculator and text.strip() else 0.0
                    for text in text_responses
                ]
                batch_empathy_scores = [
                    reward_calculator.compute_empathy_reward(text) if reward_calculator and text.strip() else 0.0
                    for text in text_responses
                ]

                # Track epoch metrics
                epoch_rewards.extend([r.item() for r in rewards])
                epoch_quality_scores.extend(batch_quality_scores)
                epoch_emotion_scores.extend(batch_emotion_scores)
                epoch_relevance_scores.extend(batch_relevance_scores)
                epoch_sentiment_scores.extend(batch_sentiment_scores)
                epoch_empathy_scores.extend(batch_empathy_scores)
                epoch_emotion_success.extend(has_emotion_flags)

                # PPO training step
                stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

                # W&B logging for this batch
                batch_metrics = {
                    "train/batch_reward": np.mean([r.item() for r in rewards]),
                    "train/batch_quality": np.mean(batch_quality_scores),
                    "train/batch_emotion": np.mean(batch_emotion_scores),
                    "train/batch_relevance": np.mean(batch_relevance_scores),
                    "train/batch_sentiment": np.mean(batch_sentiment_scores),
                    "train/batch_empathy": np.mean(batch_empathy_scores),
                    "train/batch_emotion_success_rate": np.mean(has_emotion_flags),
                    "train/epoch": epoch,
                    "train/batch": batch_idx,
                }

                # Log PPO stats if available
                if stats:
                    for key, value in stats.items():
                        if isinstance(value, (int, float, torch.Tensor)):
                            if isinstance(value, torch.Tensor):
                                value = value.item()
                            batch_metrics[f"ppo/{key}"] = value

                wandb.log(batch_metrics, step=global_step)

                # Log sample responses periodically
                if batch_idx % 10 == 0:
                    log_sample_conversations(text_responses, batch_user_inputs, batch_idx, epoch)

                # Progress logging
                if batch_idx % 5 == 0:
                    avg_reward = np.mean([r.item() for r in rewards])
                    avg_quality = np.mean(batch_quality_scores)
                    avg_emotion = np.mean(batch_emotion_scores)
                    avg_relevance = np.mean(batch_relevance_scores)
                    avg_sentiment = np.mean(batch_sentiment_scores)
                    avg_empathy = np.mean(batch_empathy_scores)
                    emotion_success_rate = np.mean(has_emotion_flags)

                    print(f"Batch {batch_idx}: Reward={avg_reward:.3f}, Quality={avg_quality:.3f}, " +
                          f"Emotion={avg_emotion:.3f}, Relevance={avg_relevance:.3f}, " +
                          f"Sentiment={avg_sentiment:.3f}, Empathy={avg_empathy:.3f}, Success={emotion_success_rate:.2%}")

                global_step += 1

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue

        # Epoch summary
        epoch_time = time.time() - epoch_start_time

        avg_epoch_reward = np.mean(epoch_rewards) if epoch_rewards else 0.0
        avg_epoch_quality = np.mean(epoch_quality_scores) if epoch_quality_scores else 0.0
        avg_epoch_emotion = np.mean(epoch_emotion_scores) if epoch_emotion_scores else 0.0
        avg_epoch_relevance = np.mean(epoch_relevance_scores) if epoch_relevance_scores else 0.0
        avg_epoch_sentiment = np.mean(epoch_sentiment_scores) if epoch_sentiment_scores else 0.0
        avg_epoch_empathy = np.mean(epoch_empathy_scores) if epoch_empathy_scores else 0.0
        emotion_success_rate = np.mean(epoch_emotion_success) if epoch_emotion_success else 0.0

        print(f"\nEpoch {epoch+1} Results:")
        print(f"Average total reward: {avg_epoch_reward:.4f}")
        print(f"Average quality score: {avg_epoch_quality:.4f}")
        print(f"Average emotion score: {avg_epoch_emotion:.4f}")
        print(f"Average relevance score: {avg_epoch_relevance:.4f}")
        print(f"Average sentiment score: {avg_epoch_sentiment:.4f}")
        print(f"Average empathy score: {avg_epoch_empathy:.4f}")
        print(f"Emotion generation success: {emotion_success_rate:.2%}")
        print(f"Epoch time: {epoch_time:.1f}s")

        # Log epoch metrics and distributions
        epoch_metrics = {
            "epoch/reward": avg_epoch_reward,
            "epoch/quality": avg_epoch_quality,
            "epoch/emotion": avg_epoch_emotion,
            "epoch/relevance": avg_epoch_relevance,
            "epoch/sentiment": avg_epoch_sentiment,
            "epoch/empathy": avg_epoch_empathy,
            "epoch/emotion_success_rate": emotion_success_rate,
            "epoch/time_seconds": epoch_time,
            "epoch/number": epoch + 1,
        }

        wandb.log(epoch_metrics, step=global_step)

        # Log score distributions
        log_score_distributions({
            "reward": epoch_rewards,
            "quality": epoch_quality_scores,
            "emotion": epoch_emotion_scores,
            "relevance": epoch_relevance_scores,
            "sentiment": epoch_sentiment_scores,
            "empathy": epoch_empathy_scores
        }, global_step)

        # Save checkpoint with tokenizer
        epoch_save_path = os.path.join(save_dir, f"epoch_{epoch+1}")
        ppo_trainer.save_pretrained(epoch_save_path)
        tokenizer.save_pretrained(epoch_save_path)  # Save tokenizer too
        print(f"Saved checkpoint to {epoch_save_path}")

        # Log model checkpoint as W&B artifact
        artifact = wandb.Artifact(f"model_epoch_{epoch+1}", type="model")
        artifact.add_dir(epoch_save_path)
        wandb.log_artifact(artifact)

        # Validation
        print("Running validation...")
        val_start_time = time.time()

        val_rewards = []
        val_quality_scores = []
        val_emotion_scores = []
        val_relevance_scores = []
        val_sentiment_scores = []
        val_empathy_scores = []
        val_emotion_success = []

        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=ppo_collator, shuffle=False)

        for val_batch_idx, val_batch in enumerate(tqdm(val_dataloader, desc="Validation", leave=False)):
            try:
                query_tensors = [item.to(device) for item in val_batch["input_ids"]]

                # Use safer indexing for validation too
                actual_batch_size = len(query_tensors)
                start_idx = val_batch_idx * batch_size
                end_idx = min(start_idx + batch_size, len(val_target_texts))
                current_batch_size = min(actual_batch_size, end_idx - start_idx)

                # Skip if we're beyond the data bounds
                if start_idx >= len(val_target_texts):
                    continue

                # Use only the valid portion
                query_tensors = query_tensors[:current_batch_size]
                batch_target_texts = val_target_texts[start_idx:start_idx + current_batch_size]
                batch_target_emotions = val_target_emotions[start_idx:start_idx + current_batch_size]
                batch_user_inputs = val_user_inputs[start_idx:start_idx + current_batch_size]
                batch_user_emotions = val_user_emotions[start_idx:start_idx + current_batch_size]

                text_responses = []
                emotion_responses = []
                has_emotion_flags = []

                for query_tensor in query_tensors:
                    _, therapist_text, therapist_emotion, has_emotion = generate_therapy_response(
                        ppo_model.pretrained_model,
                        tokenizer,
                        query_tensor,
                        device
                    )
                    text_responses.append(therapist_text)
                    emotion_responses.append(therapist_emotion)
                    has_emotion_flags.append(has_emotion)

                rewards = compute_comprehensive_rewards_enhanced(
                    text_responses,
                    emotion_responses,
                    has_emotion_flags,
                    batch_target_texts,
                    batch_target_emotions,
                    batch_user_inputs,
                    batch_user_emotions,
                    similarity_model,
                    reward_calculator,
                    device
                )

                # Compute component scores
                batch_quality_scores = [compute_text_quality_score_enhanced(text) for text in text_responses]
                batch_emotion_scores = [
                    compute_emotion_score_enhanced(has_emotion, emotion, target_emotion)
                    for has_emotion, emotion, target_emotion in zip(has_emotion_flags, emotion_responses, batch_target_emotions)
                ]
                batch_relevance_scores = [
                    compute_contextual_relevance_enhanced(text, user_input, similarity_model, device)
                    for text, user_input in zip(text_responses, batch_user_inputs)
                ]
                batch_sentiment_scores = [
                    reward_calculator.compute_sentiment_reward(text) if reward_calculator and text.strip() else 0.0
                    for text in text_responses
                ]
                batch_empathy_scores = [
                    reward_calculator.compute_empathy_reward(text) if reward_calculator and text.strip() else 0.0
                    for text in text_responses
                ]

                val_rewards.extend([r.item() for r in rewards])
                val_quality_scores.extend(batch_quality_scores)
                val_emotion_scores.extend(batch_emotion_scores)
                val_relevance_scores.extend(batch_relevance_scores)
                val_sentiment_scores.extend(batch_sentiment_scores)
                val_empathy_scores.extend(batch_empathy_scores)
                val_emotion_success.extend(has_emotion_flags)

            except Exception as e:
                print(f"Error in validation batch: {e}")
                continue

        val_time = time.time() - val_start_time

        val_avg_reward = np.mean(val_rewards) if val_rewards else 0.0
        val_avg_quality = np.mean(val_quality_scores) if val_quality_scores else 0.0
        val_avg_emotion = np.mean(val_emotion_scores) if val_emotion_scores else 0.0
        val_avg_relevance = np.mean(val_relevance_scores) if val_relevance_scores else 0.0
        val_avg_sentiment = np.mean(val_sentiment_scores) if val_sentiment_scores else 0.0
        val_avg_empathy = np.mean(val_empathy_scores) if val_empathy_scores else 0.0
        val_emotion_success_rate = np.mean(val_emotion_success) if val_emotion_success else 0.0

        print(f"Validation Results:")
        print(f"Average reward: {val_avg_reward:.4f}")
        print(f"Average quality: {val_avg_quality:.4f}")
        print(f"Average emotion: {val_avg_emotion:.4f}")
        print(f"Average relevance: {val_avg_relevance:.4f}")
        print(f"Average sentiment: {val_avg_sentiment:.4f}")
        print(f"Average empathy: {val_avg_empathy:.4f}")
        print(f"Emotion generation success: {val_emotion_success_rate:.2%}")
        print(f"Validation time: {val_time:.1f}s")

        # Log validation metrics
        val_metrics = {
            "val/reward": val_avg_reward,
            "val/quality": val_avg_quality,
            "val/emotion": val_avg_emotion,
            "val/relevance": val_avg_relevance,
            "val/sentiment": val_avg_sentiment,
            "val/empathy": val_avg_empathy,
            "val/emotion_success_rate": val_emotion_success_rate,
            "val/time_seconds": val_time,
        }

        wandb.log(val_metrics, step=global_step)

        # Log validation score distributions
        log_score_distributions({
            "val_reward": val_rewards,
            "val_quality": val_quality_scores,
            "val_emotion": val_emotion_scores,
            "val_relevance": val_relevance_scores,
            "val_sentiment": val_sentiment_scores,
            "val_empathy": val_empathy_scores
        }, global_step)

    # Save final model with tokenizer
    final_save_path = os.path.join(save_dir, "final_model")
    ppo_trainer.save_pretrained(final_save_path)
    tokenizer.save_pretrained(final_save_path)  # Save tokenizer too

    # Log final model as artifact
    final_artifact = wandb.Artifact("final_model", type="model")
    final_artifact.add_dir(final_save_path)
    wandb.log_artifact(final_artifact)

    total_time = time.time() - start_time
    print(f"\nTraining completed! Total time: {total_time:.1f}s")
    print(f"Final model saved to {final_save_path}")
    print(f"W&B run url: {wandb.run.url}")

    # Log final summary
    wandb.log({
        "training/total_time_seconds": total_time,
        "training/total_steps": global_step,
        "training/epochs_completed": num_epochs
    })

    # Finish W&B run
    wandb.finish()

    return ppo_trainer

if __name__ == "__main__":
    train_dataset = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/train.json"
    val_dataset = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/val.json"
    sft_model = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt"
    tokenizer_path = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"

    try:
        ppo_model = run_ppo_training_with_wandb(
            train_data_path=train_dataset,
            val_data_path=val_dataset,
            sft_model_path=sft_model,
            tokenizer_path=tokenizer_path,
            num_epochs=8,
            batch_size=16,
            save_dir="/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model",
            project_name="8m19therapy-chatbot-ppo-enhanced",
            run_name="8m19therapy_ppo_enhanced_rewards"
        )

        print("PPO training with enhanced reward components completed successfully!")

    except Exception as e:
        print(f"Training failed: {e}")
        import traceback
        traceback.print_exc()
        raise

Starting PPO training with Weights & Biases integration...


ppo/objective/entropy,▁▁▁▁▁▂▁▁▁▁▁▁▂▂▂▂▂███▇▇▆▆▄▄▄▃▃▃▃▂▃▂▃▂▃▂▃▂
ppo/objective/kl,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ppo/objective/kl_coef,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁
ppo/ppo/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ppo/ppo/loss/policy,▇▃▃▃▃▁▂▁▃▂▂▂▂▃▂▁▁▁▁▂▄▂▃▃▃▄▅▅▆▅█▆▆▆▆▅▅▄▆▆
ppo/ppo/loss/total,█▇▅▄▄▃▃▃▃▂▃▅▃▂▂▂▃▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▂▁▁▁▂▁
ppo/ppo/loss/value,▂▇█▆▄▃▃▃▃▅▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▂▁▂▁▂▂▁▂▁▁▂▁▂
ppo/ppo/mean_non_score_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ppo/ppo/mean_scores,▅▅▃▄▅▆▇█▇▆▆▆▇▆▇▅▇▂▂▁▁▁▂▁▂▂▁▂▂▁▂▃▂▂▂▂▃▂▃▃
ppo/ppo/policy/advantages_mean,▁▃▄▁▅▅▄▅▅▅▄▅▃▅▄▄▄▅▅▅▄█▅▅▅▅▅▄▄▄▄▄▄▅▄▄▄▄▄▄
ppo/ppo/policy/approxkl,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁█▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁


W&B run: 8m19therapy_ppo_enhanced_rewards
W&B url: https://wandb.ai/ericzhangez1006-ucl/8m19therapy-chatbot-ppo-enhanced/runs/agsy9y6e
Loading tokenizer...
Tokenizer vocabulary size: 50269
Loading base model...
Loading SFT checkpoint from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt...
SFT checkpoint loaded successfully
Loading similarity model...
Similarity model loaded successfully
Loading sentiment and empathy models...
Sentiment and empathy models loaded successfully
Creating PPO configuration...
Creating PPO models...
Loading datasets...


/usr/local/lib/python3.12/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


Loaded tokenizer from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer


...

Validation Results:
Average reward: 2.1384
Average quality: -0.6427
Average emotion: -0.0180
Average relevance: 0.3618
Average sentiment: 0.2897
Average empathy: 0.8862
Emotion generation success: 73.93%
Validation time: 297.8s


wandb: Adding directory to artifact (/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/final_model)... Done. 1.6s



Training completed! Total time: 16402.2s
Final model saved to /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/final_model
W&B run url: https://wandb.ai/ericzhangez1006-ucl/8m19therapy-chatbot-ppo-enhanced/runs/agsy9y6e


distributions/emotion_mean,▆███▆▃▂▁
distributions/emotion_std,▄▂▁▁▄▇██
distributions/empathy_mean,▁▄▄▅▅▆▇█
distributions/empathy_std,█▃▄▂▃▁▃▁
distributions/quality_mean,▇█▇█▆▅▂▁
distributions/quality_std,▂▁▂▁▃▅▇█
distributions/relevance_mean,▁▄▆█▇▆▅▃
distributions/relevance_std,▆▄▄▁▂▄▇█
distributions/reward_mean,▃▆▇█▆▃▂▁
distributions/reward_std,▄▂▂▁▃▆▇█
distributions/sentiment_mean,██▇▇▆▂▁▁


PPO training with enhanced reward components completed successfully!
